In [ ]:
import sys
import os
import yaml
import numpy as np
from scipy import stats

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
pd.options.mode.chained_assignment = None

sys.path.append('../python/')

import logging
logging.basicConfig(level=logging.DEBUG, filename="dash_logs.log")

In [ ]:
dict_parameters = yaml.safe_load(open("../parameters/july_2024_data_parameters.yaml", "r"))

In [ ]:
LIST_ANALYTES_TO_CHECK = list(set(dict_parameters["list of analytes"]) - set(["IFN-gamma", "IP-10", "IL-17"]))

In [ ]:
pd_df_estimated_concentrations = pd.read_csv(
    open(
        os.path.join(
            dict_parameters["output directory path"],
            dict_parameters["estimated concentrations file name"]
        ),
        "rb"
    )
)

In [ ]:
pd_df_quality_control_concentrations = pd.read_csv(
    open(
        os.path.join(
            dict_parameters["data directory path"],
            dict_parameters["quality control concentrations file name"]
        ),
        "rb"
    )
)

In [ ]:
pd_df_quality_control_concentrations

In [ ]:
pd_df_estimated_concentrations

In [ ]:
pd_df_estimated_qc_concentrations = (
    pd_df_estimated_concentrations[
        pd_df_estimated_concentrations["sample name annotations"]
        .str.contains("QC-\d+", regex = True)
    ]
)

In [ ]:
pd_df_estimated_qc_concentrations

In [ ]:
columns_to_keep = ["sample name annotations", "plate number"]
for str_analyte in LIST_ANALYTES_TO_CHECK:
    columns_to_keep.append(f"estimated concentration {str_analyte}")
pd_df_estimated_qc_concentrations = pd_df_estimated_qc_concentrations[columns_to_keep]

In [ ]:
pd_df_estimated_qc_concentrations

In [ ]:
pd_df_estimated_qc_concentrations = pd_df_estimated_qc_concentrations.rename(
    columns = {f"estimated concentration {str_analyte}": str_analyte for str_analyte in LIST_ANALYTES_TO_CHECK}
)

In [ ]:
pd_df_estimated_qc_concentrations

In [ ]:
pd_df_melted_estimated_qc_concentrations = pd_df_estimated_qc_concentrations.melt(
    id_vars = ["sample name annotations", "plate number"],
    var_name = "analyte",
    value_name = "concentration"
)

In [ ]:
pd_df_melted_estimated_qc_concentrations

In [ ]:
pd_df_melted_quality_control_concentrations = pd_df_quality_control_concentrations.melt(
    id_vars = ["QC name"], 
    var_name = "analyte bound",
    value_name = "concentration"
)

In [ ]:
pd_df_melted_quality_control_concentrations

In [ ]:
pd_df_melted_quality_control_concentrations[["analyte", "bound"]] = pd_df_melted_quality_control_concentrations["analyte bound"].str.split(" ", expand = True)

In [ ]:
pd_df_melted_quality_control_concentrations = pd_df_melted_quality_control_concentrations.drop(columns = ["analyte bound"])

In [ ]:
pd_df_melted_quality_control_concentrations

In [ ]:
pd_df_pivoted_quality_control_concentrations = pd_df_melted_quality_control_concentrations.pivot(
    index = ["QC name", "analyte"],
    columns = "bound",
    values = "concentration"
).reset_index().rename(columns = {"bound": "analyte", "QC name": "sample name annotations"})

In [ ]:
pd_df_pivoted_quality_control_concentrations

In [ ]:
pd_df_estimates_and_bounds = pd.merge(
    pd_df_melted_estimated_qc_concentrations,
    pd_df_pivoted_quality_control_concentrations,
    on = ["sample name annotations", "analyte"],
    how = "inner"
)

In [ ]:
pd_df_estimates_and_bounds

In [ ]:
# def check_value_within_qc_bounds(pd_row, str_col_name, low_vals, high_vals):
#     qc_number = int(pd_row["sample name annotations"].split("-")[1]) - 1
#     if (pd_row[str_col_name] >= low_vals[qc_number]) & (pd_row[str_col_name] <= high_vals[qc_number]):
#         return (
#             'within range', 
#             low_vals[qc_number], 
#             high_vals[qc_number]
#         )
#     elif (pd_row[str_col_name] < low_vals[qc_number]):
#         return (
#             f'below range by {low_vals[qc_number] - pd_row[str_col_name]:.2f}', 
#             low_vals[qc_number], 
#             high_vals[qc_number]
#         )
#     else:
#         return (
#             f'above range by {pd_row[str_col_name] - high_vals[qc_number]:.2f}', 
#             low_vals[qc_number], 
#             high_vals[qc_number]
#         )

In [ ]:
# for str_analyte in dict_parameters["list of analytes"]:
#     low_vals = []
#     high_vals = []
#     for i in range(1, 3):
#         low_vals.append(
#             pd_df_quality_control_concentrations[
#                 pd_df_quality_control_concentrations["QC name"] == f"QC-{i}"
#             ][f"{str_analyte} low"].values[0]
#         )
#         high_vals.append(
#             pd_df_quality_control_concentrations[
#                 pd_df_quality_control_concentrations["QC name"] == f"QC-{i}"
#             ][f"{str_analyte} high"].values[0]
#         )
# 
#     pd_df_estimated_qc_concentrations[[f'{str_analyte} check', f'{str_analyte} low', f'{str_analyte} high']] = (
#         pd_df_estimated_qc_concentrations.apply(
#             lambda row: check_value_within_qc_bounds(
#                 row, 
#                 f"estimated concentration {str_analyte}",
#                 low_vals, 
#                 high_vals
#             ), axis=1, result_type = 'expand'
#         )
#     )

In [ ]:
# column_order = ["sample name annotations", "plate number"]
# for str_analyte in LIST_ANALYTES_TO_CHECK:
#     column_order.append(f"estimated concentration {str_analyte}")
#     column_order.append(f"{str_analyte} check")
#     column_order.append(f"{str_analyte} low")
#     column_order.append(f"{str_analyte} high")
# pd_df_estimated_qc_concentrations = pd_df_estimated_qc_concentrations[column_order]

In [ ]:
pd_df_estimates_and_bounds

In [ ]:
dash_app_object_boxes = dash.Dash(__name__)

# Define the layout
dash_app_object_boxes.layout = html.Div([
    html.H1("Plots of estimated concentrations"),

    html.Div([
        html.Label("Plate:"),
        dcc.Dropdown(
            id='plate-dropdown',
            options=[
                {'label': plate_number, 'value': plate_number} for plate_number in dict_parameters["plate number to file associations"].keys()
            ],
            value=list(dict_parameters["plate number to file associations"].keys())[0]
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("QC label:"),
        dcc.Dropdown(
            id='qc-label-dropdown',
            options=[
                {'label': 'QC-1', 'value': 'QC-1'},
                {'label': 'QC-2', 'value': 'QC-2'},
            ],
            value='QC-1',
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),

    dcc.Graph(id='scatter-plot')
    
], style={'backgroundColor': 'white', 'padding': '20px'})


@dash_app_object_boxes.callback(
    Output('scatter-plot', 'figure'),
    Input('plate-dropdown', 'value'),
    Input('qc-label-dropdown', 'value'),
)
def update_graph(int_plate_number, str_qc_label):
    
    logging.debug(int_plate_number)
    logging.debug(str_qc_label)
    logging.debug(pd_df_estimates_and_bounds)
    
    pd_df_data_to_plot = pd_df_estimates_and_bounds[
        pd_df_estimates_and_bounds["plate number"] == int(int_plate_number)
    ]
    pd_df_data_to_plot = pd_df_data_to_plot[
        pd_df_data_to_plot["sample name annotations"].str.contains(str(str_qc_label))
    ]
    
    logging.debug(pd_df_data_to_plot)
    
    fig = go.Figure()
    fig.add_trace(
        go.Box(
            x = pd_df_data_to_plot["analyte"], 
            y = pd_df_data_to_plot["low"],
            boxpoints = 'all',
            pointpos=0,
            fillcolor='rgba(255,255,255,0)',
            line=dict(color='rgba(255,255,255,0)'),
            marker=dict(color='green'),
        )
    )
    fig.add_trace(
        go.Box(
            x = pd_df_data_to_plot["analyte"], 
            y = pd_df_data_to_plot["high"],
            boxpoints = 'all',
            pointpos=0,
            fillcolor='rgba(255,255,255,0)',
            line=dict(color='rgba(255,255,255,0)'),
            marker=dict(color='red'),
        )
    )
    fig.add_trace(
        go.Box(
            x = pd_df_data_to_plot["analyte"], 
            y = pd_df_data_to_plot["concentration"],
            boxpoints = 'all',
            pointpos=0,
            fillcolor='rgba(255,255,255,0)',
            line=dict(color='rgba(255,255,255,0)'),
            marker=dict(color='blue'),
        )
    )
    
    

    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis_title="analyte",
        yaxis_title="concentration",
        font=dict(
            family="Arial",
            size=16,
            color="black"
        )
    )
    return fig

# Run the app
if __name__ == '__main__':
    dash_app_object_boxes.run(jupyter_mode="inline", debug = True, port=3358)